In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:08<00:00, 43.3MB/s]
100% 1.06G/1.06G [00:08<00:00, 138MB/s] 


In [3]:
import zipfile
zip_ref=zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/dogs-vs-cats.zip' mode='r'>>

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [10]:
#using generator
train_ds =keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred' ,
    label_mode = 'int' ,
    batch_size=32,
    image_size=(256,256)
)

test_ds =keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred' ,
    label_mode = 'int' ,
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [11]:
#Normalization
def process(image,label):
  image  =tf.cast(image/255. ,tf.float32)
  return image,label

train_ds =train_ds.map(process)
test_ds =test_ds.map(process)




In [14]:
#create CNN model

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation = 'relu' ,input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation = 'relu' ,input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation = 'relu' ,input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [15]:
model.summary();

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)      

In [16]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])




In [18]:
history= model.fit(train_ds,epochs=10,validation_data=test_ds)

Epoch 1/10
625/625 [==============================] - 3329s 5s/step - loss: 0.5841 - accuracy: 0.6846 - val_loss: 0.5044 - val_accuracy: 0.7544
Epoch 2/10
625/625 [==============================] - 3226s 5s/step - loss: 0.4642 - accuracy: 0.7797 - val_loss: 0.4434 - val_accuracy: 0.7992
Epoch 3/10
625/625 [==============================] - ETA: 0s - loss: 0.3686 - accuracy: 0.8355

KeyboardInterrupt: 